In [160]:
import requests
import json
import pandas as pd
import time
import pickle
import datetime
import re
from bs4 import BeautifulSoup
from IPython.display import clear_output # Cell output clear를 위함

In [161]:
df_company_list = pd.read_csv('Target_company_list.csv', encoding='utf-8', dtype={'stock_code': object})
df_company_list

,stock_code,company_name,market,bankruptcy,delisted_check,delisted_date
0,008670,굿모닝신한증권,Y,0,1,2005-01-05
1,005600,중앙제지,Y,1,1,2005-01-06
2,035780,그로웰텔레콤,K,0,1,2005-01-11
3,035910,현대멀티캡,K,1,1,2005-01-19
4,009220,그로웰전자,K,0,1,2005-01-19
...,...,...,...,...,...,...
2703,000070,삼양홀딩스,Y,0,0,NaN
2704,000060,메리츠화재,Y,0,0,NaN
2705,000050,경방,Y,0,0,NaN
2706,000040,KR모터스,Y,0,0,NaN


In [162]:
url_prefix = 'http://dart.fss.or.kr/api/search.json?auth='
auth = 'dfb3cef9547d718553835b595412647b5001e090'
headers = {'user-agent': 'Mozilla/5.0'}

In [163]:
def create_reportDict_audit(auth, df_company_list):
    try:
        with open('report_url_Dict_audit.pickle', 'rb') as f:
            report_url_Dict = pickle.load(f)
    except FileNotFoundError:
        report_url_Dict = {}
    
    last_progress = len(report_url_Dict)
        
    for index in df_company_list.index:
        if last_progress >= (index+1):
            continue
            
        company_info = df_company_list.iloc[index]
        stock_code = company_info['stock_code']
        
        report_url_Dict[stock_code] = {}
        
        url = url_prefix + auth +\
         '&crp_cd=' + stock_code + \
         '&bsn_tp=F001' + \
         '&start_dt=20010101&page_set=100'
        
        
        req = requests.get(url, headers=headers)
        json_data = req.json() # 읽을 수 있게 디코딩
        check_num = 1
        
        report_url_Dict[stock_code] = {
            'A-1' : None
            ,'A-2' : None
            ,'A-3' : None
            ,'A-4' : None
            ,'A-5' : None
            ,'A-6' : None
            ,'A-7' : None
            ,'A-8' : None
            ,'A-9' : None
            ,'A-10' : None
            }

        if str(company_info['delisted_date']) == 'nan':
            for each in json_data['list']:
                if (check_num < 11):
                    if ('정정' not in each['rpt_nm']):
                        report_url_Dict[stock_code]['A-' + str(check_num)] = [each['rpt_nm'], 'http://dart.fss.or.kr/dsaf001/main.do?rcpNo=' + each['rcp_no']]
                        check_num += 1
                else:
                    break
        else:
            for each in json_data['list']:
                if (check_num < 11):
                    if ('정정' not in each['rpt_nm']):
                        if datetime.datetime.strptime(each['rcp_dt'], '%Y%m%d') > datetime.datetime.strptime(company_info['delisted_date'],'%Y-%m-%d'):
                            continue
                        else:
                            if check_num != 1:
                                if each['rpt_nm'][-9:-1] == report_url_Dict[stock_code]['A-' + str(check_num-1)][0][-9:-1]:
                                    continue
                            report_url_Dict[stock_code]['A-' + str(check_num)] = [each['rpt_nm'], 'http://dart.fss.or.kr/dsaf001/main.do?rcpNo=' + each['rcp_no']]
                            check_num += 1
                else:
                    break
        
        #print(json_data)
        print(company_info['company_name'], company_info['stock_code'])
        for each in report_url_Dict[stock_code]:
            print(each, report_url_Dict[stock_code][each])
        print('-------------')
        clear_output(wait=True)
        with open('report_url_Dict_audit.pickle', 'wb') as f:
            pickle.dump(report_url_Dict, f)
        
        time.sleep(1)

In [164]:
create_reportDict_audit(auth, df_company_list)

동화약품 000020
A-1 ['감사보고서 (2009.12)', 'http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20100329000247']
A-2 ['감사보고서 (2009.03)', 'http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20090602000198']
A-3 ['감사보고서 (2008.03)', 'http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20080612000114']
A-4 ['감사보고서 (2007.03)', 'http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20070530000185']
A-5 ['감사보고서 (2006.03)', 'http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20060529000143']
A-6 ['감사보고서 (2005.03)', 'http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20050608000061']
A-7 ['감사보고서 (2004.03)', 'http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20040601000086']
A-8 ['감사보고서 (2003.03)', 'http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20030603000007']
A-9 ['감사보고서 (2002.03)', 'http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20020527000077']
A-10 ['감사보고서 (2001.03)', 'http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20010601000098']
-------------


In [165]:
with open('report_url_Dict_audit.pickle', 'rb') as f:
    report_url_Dict_audit = pickle.load(f)
    print(len(report_url_Dict_audit))
    print(report_url_Dict_audit['008670'])

2708
{'A-1': ['감사보고서 (2004.03)', 'http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20040603000224'], 'A-2': ['감사보고서 (2003.03)', 'http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20030611000117'], 'A-3': ['감사보고서 (2002.03)', 'http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20020607000050'], 'A-4': ['감사보고서 (2001.03)', 'http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20010531000001'], 'A-5': None, 'A-6': None, 'A-7': None, 'A-8': None, 'A-9': None, 'A-10': None}


In [167]:
def parsing_reportpage_code(url): # 감사보고서 본문에 직접 접근하는 URL 추출
    req_each = requests.get(url, headers=headers)
    html = req_each.text
    soup = BeautifulSoup(html, 'html.parser')
    
    script_total = soup.find_all('script', type='text/javascript')
    for each_element in script_total:
        if '감사보고서' in each_element.text:
            target_element = each_element
            break
    
    p = re.compile("\({(.*?)}\)", re.DOTALL)
    p2 = re.compile("viewDoc\((.*?)\)")
    audit_body_url = ''
    for each_regex_result in p.findall(target_element.text):
        cleansed_result = each_regex_result.replace('\n','').replace('\t','').replace('\\','')
        if '감사보고서' in cleansed_result:
            list_of_body_url_fragment = p2.findall(cleansed_result)[0].replace("'","").replace(' ','').split(',')
            audit_body_url = 'http://dart.fss.or.kr/report/viewer.do?rcpNo=' + list_of_body_url_fragment[0] + '&dcmNo=' + list_of_body_url_fragment[1] + '&eleId=' + list_of_body_url_fragment[2] +'&offset=' + list_of_body_url_fragment[3] +'&length=' + list_of_body_url_fragment[4] + '&dtd=' + list_of_body_url_fragment[5]
    return audit_body_url

In [168]:
def get_audit_report_body(body_url): # 감사보고서 본문 내용을 추출
    req_each = requests.get(body_url, headers=headers)
    html = req_each.text
    soup = BeautifulSoup(html, 'html.parser')
    l = []
    
    for each in soup.find_all('p'):    
        l.append(each.text)

    return ''.join(l)

In [177]:
get_audit_report_body(parsing_reportpage_code('http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20030320000330')).replace('\xa0',' ')

'외부감사인의 감사보고서    본 감사인은 대한제당 주식회사의 2002년 12월 31일과 2001년 12월 31일 현재의대차대조표와 동일로 종료되는 양회계년도의 손익계산서, 이익잉여금처분계산서 및현금흐름표를 감사하였습니다. 이 재무제표를 적정하게 작성할 책임은 회사경영자 에게 있으며 본 감사인의 책임은 동 재무제표에 대하여 감사를 실시하고 이를 근거로이 재무제표에 대하여 의견을 표명하는데 있습니다.     본 감사인은 한국의 회계감사기준에 따라 감사를 실시하였습니다. 이 기준은 본 감사인이 재무제표가 중대하게 왜곡표시되지 아니하였다는 것을 합리적으로 확신하도록 감사를 계획하고 실시할 것을 요구하고 있습니다. 감사는 재무제표상의 금액과 공시내용을 뒷받침하는 감사증거에 대하여 시사의 방법을 적용하여 검증하는 것을 포함하고 있습니다. 또한 감사는 재무제표의 전반적인 표시내용에 대한 평가뿐만 아니라 회사가 적용한 회계원칙과 경영자에 의한 중요한 추정에 대해 평가하는 것을 포함하고 있습니다. 본 감사인이 실시한 감사가 감사의견 표명을 위한 합리적인 근거를제공하고 있다고 본 감사인은 믿습니다.    본 감사인의 의견으로는 상기 재무제표는 대한제당 주식회사의 2002년과 2001년12월 31일 현재의 재무상태와 동일로 종료되는 양회계연도의 경영성과 그리고 이익잉여금의 변동과 현금흐름의 내용을 한국의 기업회계기준에 따라 중요성의 관점에서적정하게 표시하고 있습니다.   다음은 감사의견에는 영향이 없지만 감사보고서 이용자의 합리적인 의사결정에 참고가 되는 사항입니다.(1) 피투자회사의 청산으로 인한 관련 자산·부채의 양수   재무제표에 대한 주석 2에서 설명하고 있는 바와 같이 회사의 피투자회사인 (주)TS센트릭이 주주총회의 결의에 따라 해산함에 따라 출자금액 회수를 위해 잔여재산을18,050백만원에 양수하였으며, 채무 754백만원과 US$ 11,857,290.89를 인수하였습니다.(2) 재고자산 평가방법의 변경   재무제표에 대한 주석 2에서 설명하고 있는 바와 같이 회사는 

In [ ]:
dict_for_df = {'stock_code':[], 'company_name':[], 'bankruptcy':[], 'report_name':[], 'url':[], 'text':[], 'report_date':[]}
# DataFrame으로 만들 Dictionary
i = 1
for each_stock_code in report_url_Dict_audit:
    print(each_stock_code, i)
    for each_report in report_url_Dict_audit[each_stock_code]:
        time.sleep(1)
        if report_url_Dict_audit[each_stock_code][each_report] == None:
            break
        dict_for_df['url'].append(report_url_Dict_audit[each_stock_code][each_report][1])
        dict_for_df['stock_code'].append(each_stock_code)
        dict_for_df['company_name'].append(df_company_list[df_company_list['stock_code'] == each_stock_code]['company_name'].values[0])
        dict_for_df['bankruptcy'].append(df_company_list[df_company_list['stock_code'] == each_stock_code]['bankruptcy'].values[0])
        dict_for_df['report_name'].append(each_report)
        dict_for_df['text'].append(get_audit_report_body(parsing_reportpage_code(report_url_Dict_audit[each_stock_code][each_report][1])).replace('\xa0',' '))
        dict_for_df['report_date'].append(datetime.datetime.strptime(report_url_Dict_audit[each_stock_code][each_report][1][44:52], '%Y%m%d'))
    i += 1

In [240]:
df = pd.DataFrame(dict_for_df)
df

,stock_code,company_name,bankruptcy,report_name,url,text,report_date
0,008670,굿모닝신한증권,0,A-1,http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20...,외부감사인의 감사보고서본 감사인은 첨부된 굿모닝신한증권주식회사의 2004년 3월 3...,2004-06-03
1,008670,굿모닝신한증권,0,A-2,http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20...,"외부감사인의 감사보고서본 감사인은 굿모닝신한증권주식회사(구, 굿모닝증권주식회사)의 ...",2003-06-11
2,008670,굿모닝신한증권,0,A-3,http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20...,외부감사인의 감사보고서본 감사인은 굿모닝증권주식회사의 2002년 3월 31일과 20...,2002-06-07
3,008670,굿모닝신한증권,0,A-4,http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20...,외부감사인의 감사보고서본 감사인은 굿모닝증권주식회사의 2001년 3월 31일과 20...,2001-05-31
4,005600,중앙제지,1,A-1,http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20...,외부감사인의 감사보고서본 감사인은 첨부된 중앙제지 주식회사의 2003년 12월 31...,2004-03-31
...,...,...,...,...,...,...,...
20084,000020,동화약품,0,A-6,http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20...,외부감사인의 감사보고서1. 감사보고서 본문 본 감사인은 첨부된 동화약품공업주식회사...,2005-06-08
20085,000020,동화약품,0,A-7,http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20...,외부감사인의 감사보고서1. 감사보고서 본문 본 감사인은 첨부된 동화약품공업주식회사...,2004-06-01
20086,000020,동화약품,0,A-8,http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20...,외부감사인의 감사보고서1. 감사보고서 본문 본 감사인은 동화약품공업주식회사의 20...,2003-06-03
20087,000020,동화약품,0,A-9,http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20...,외부감사인의 감사보고서1. 감사보고서 본문 본 감사인은 동화약품공업주식회사의 20...,2002-05-27


In [241]:
df.to_csv('audit_report_df.csv')